In [7]:
#hide
#skip
%config Completer.use_jedi = False
# upgrade fastrl on colab
! [ -e /content ] && pip install -Uqq fastrl['dev'] pyvirtualdisplay && \
                     apt-get install -y xvfb python-opengl > /dev/null 2>&1 
# NOTE: IF YOU SEE VERSION ERRORS, IT IS SAFE TO IGNORE THEM. COLAB IS BEHIND IN SOME OF THE PACKAGE VERSIONS

In [9]:
# hide
from fastcore.imports import in_colab

# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.showdoc import *
    from nbdev.imports import *
    if not os.environ.get("IN_TEST", None):
        assert IN_NOTEBOOK
        assert not IN_COLAB
        assert IN_IPYTHON

In [50]:
# export
# Python native modules
import os,warnings,logging
# Third party libs
from fastcore.all import *
from fastai.torch_core import *
from fastai.basics import *
from torch.utils.tensorboard import SummaryWriter
import pandas as pd
import torch
import numpy as np
# Local modules

_logger = logging.getLogger()
_li = _logger.info

In [11]:
# default_exp core

# Core
> Core libs for fastrl

In [48]:
# export
@call_parse
def fastrl_make_requirements(
    path:Path=None, # The path to a dir with the settings.ini, if none, cwd.
    project_file:str='settings.ini', # The file to load for reading the requirements
    out_path:Path=None # The output path (can be relative to `path`)
):
    logging.basicConfig()
    requirement_types = ['','dev_','pip_']
    path = ifnone(path, Path.cwd())/project_file

    if not path.exists(): raise OSError(f'File {path} does not exist')

    out_path = ifnone(out_path, Path('extra'))
    out_path = out_path if out_path.is_absolute() else path.parent/out_path
    out_path.mkdir(parents=True, exist_ok=True)
    _li('Outputting to path: %s',out_path)
    config = Config(path.parent,path.name)

    for req in requirement_types:
        requirements = config[req+'requirements']

        requirements = requirements.replace(' ','\n')

        Path(out_path/(req+'requirements.txt')).write_text(requirements)

## D
> A better dictionary

In [13]:
# export
def map_dict_ex(d,f,*args,gen=False,wise=None,**kwargs):
    "Like `map`, but for dicts and uses `bind`, and supports `str` and indexing"
    g = (bind(f,*args,**kwargs) if callable(f)
         else f.format if isinstance(f,str)
         else f.__getitem__)

    if wise is None:  return map(g,d.items())
    return ((k,g(v)) if wise=='value' else (g(k),v) for k,v in d.items())

Check that general mapping for dicts works nicely...

In [14]:
test_dict={'a':1,'b':2,'c':3}
test_eq(dict(map_dict_ex(test_dict,lambda t:(t[0]+'_new',t[1]+1))),{'a_new':2,'b_new':3,'c_new':4})

Check that key and value wise mapping works correctly...

In [15]:
test_eq(dict(map_dict_ex(test_dict,lambda k:k+'_new',wise='key')),{'a_new':1,'b_new':2,'c_new':3})
test_eq(dict(map_dict_ex(test_dict,lambda v:v+1,wise='value')),{'a':2,'b':3,'c':4})

In [16]:
# export
_error_msg='Found idxs: %s have values more than %s e.g.: %s'

class D(dict):
    "Improved version of `dict` with array handling abilities"
    def __init__(self,*args,mapping=False,**kwargs):
        self.mapping=mapping
        super().__init__(*args,**kwargs)
        
    def eq_k(self,o:'D',with_diff=False):
        eq=set(o.keys())==set(self.keys())
        if with_diff: return eq,set(o.keys()).symmetric_difference(set(self.keys()))
        return eq
    def _new(self,*args,**kwargs): return type(self)(*args,**kwargs)
    
    def map(self,f,*args,gen=False,**kwargs): 
        return (self._new,noop)[gen](map_dict_ex(self,f,*args,**kwargs),mapping=True)
    def mapk(self,f,*args,gen=False,wise='key',**kwargs):
        return self.map(f,*args,gen=gen,wise=wise,**kwargs)
    def mapv(self,f,*args,gen=False,wise='value',**kwargs):
        return self.map(f,*args,gen=gen,wise=wise,**kwargs)

In [17]:
test_dict=D({'a':1,'b':2,'c':3})
test_eq(test_dict.map(lambda t:(t[0]+'_new',t[1]+1)),{'a_new':2,'b_new':3,'c_new':4})
test_eq(isinstance(test_dict.map(lambda t:(t[0]+'_new',t[1]+1),gen=True),map),True)
test_eq(dict(test_dict.map(lambda t:(t[0]+'_new',t[1]+1),gen=True)),{'a_new':2,'b_new':3,'c_new':4})

test_eq(test_dict.mapk(lambda k:k+'_new'),{'a_new':1,'b_new':2,'c_new':3})
test_eq(dict(test_dict.mapk(lambda k:k+'_new',gen=True)),{'a_new':1,'b_new':2,'c_new':3})

test_eq(test_dict.mapv(lambda v:v+1,wise='value'),{'a':2,'b':3,'c':4})
test_eq(dict(test_dict.mapv(lambda v:v+1,gen=True,wise='value')),{'a':2,'b':3,'c':4})

`BD` is the primary data structure that `fastrl` uses. It allows for easily iterating and doing operations on steps attained from environments.

## BD 
> A batch wise dictionary that requires all values to be numpy,tensor, or None.

We need to change any indexer that is passed. We don't know if the indexer is going to
be a numpy array, slice, tensor, or int.
All we know is 2 things:
- If it is an int, the batch dim will disappear
- If it is an indexer, then the batch dim will stay, but be smaller

In [18]:
# export
def tensor2shape(k,t:'TensorBatch',relative_shape=False):
    "Converts a tensor into a dict of shapes, or a 1d numpy array"
    return {
        k:t.cpu().numpy().reshape(-1,) if len(t.shape)==2 and t.shape[1]==1 else 
        [str((1,*t.shape[1:]) if relative_shape else t.shape)]*t.shape[0]
    }

`tensor2shape` is a function for preparing tensors for showing in pandas. For example
if we have a tensor that has 5 dimensions, it would be very hard to read if displayed in pandas

In [19]:
test_eq(tensor2shape('test',torch.randn(3,5)),
       {'test': ['torch.Size([3, 5])', 'torch.Size([3, 5])', 'torch.Size([3, 5])']})

If the tensor has only 1 channel, then we can show its literal value...

In [20]:
test_eq(tensor2shape('test',torch.tensor([[1],[2],[3]]))['test'],
        {'test': np.array([1, 2, 3])}['test'])

In [21]:
# export
def tensor2mu(k,t:Tensor): return {f'{k}_mu':t.reshape(t.shape[0],-1).double().mean(axis=1)}
tensor2mu.__docs__="Returns a dict with key `k`_mu with the mean of `t` batchwise "

Outputs a dictionary that has the mean of the tensor. The returned dictionary's keys 
have the naming convention: *[k]_mu*.

In [22]:
o=torch.randn(3,5)
test_eq(tensor2mu('test',o)['test_mu'],{'test_mu': o.double().mean(axis=1)}['test_mu'])

Ok I have reworked the tensor management soooo many times. I think the core issue is the tensors themselves. They should individually be incharge of
determining if they match the expected batch size I think....

In [23]:
# export
class TensorBatch(TensorBase):
    "A tensor assumes a batch dimension"
    def __new__(cls, x, bs=1,**kwargs):
        res=super(TensorBatch,cls).__new__(cls,x,**kwargs)
        setattr(res,'_bs',x.bs() if isinstance(x,cls) else bs)
        return res
    
    def strict(self):
        assert self.shape[0]==self._bs,f'Tensor has shape {self.shape} while bs is {self._bs}'
        
    def bs(self): return self.shape[0]
    def get(self,*args):
        "Get a possible subset of a tensor while maintaining a batch dim."
        res=self[args]
        if len(self.shape)>len(res.shape): res=res.unsqueeze(0)
        return res
    
    @classmethod
    def vstack(cls,*args): 
        return cls(torch.vstack(*args),bs=L(*args).map(cls).map(Self.bs()).sum())
              
def obj2tensor(o):
    return (o if isinstance(o,TensorBatch) else
            TensorBatch(np.array(o)) if isinstance(o,(L,list)) else
            TensorBatch(o) if isinstance(o,(np.ndarray,Tensor,TensorBatch)) else
            TensorBatch([o])) 

def _get_bs(o): return o.bs if isinstance(o,TensorBatch) else TensorBatch(o).bs

# export
class BD(D):
    def __init__(self,*args,**kwargs):
        super().__init__(*args,**kwargs)
        self.bs=list(self.values())[0].bs
        
    def __radd__(self,o): return self if isinstance(o,int) else self.__add__(o) 
    def __add__(self,o):
        return BD({k:TensorBatch.vstack((self[k],o[k])) for k in self})
    
    def __getitem__(self,o):
        if is_listy(o) or isinstance(o,(TensorBatch,int,Tensor)): 
            return type(self)({k:self[k].get(o) for k in self})
        return super().__getitem__(o)

    @classmethod
    def merge(cls,*ds,**kwargs): return cls(merge(*ds),**kwargs)
    @delegates(pd.DataFrame)
    def pandas(self,mu=False,relative_shape=False,jupyter_nrows=None,**kwargs):
        "Turns a `BD` into a pandas Dataframe optionally showing `mu` of values."
        if jupyter_nrows is not None: pd.set_option('display.max_rows', jupyter_nrows)
        return pd.DataFrame(merge(
            *tuple(tensor2shape(k,v,relative_shape) for k,v in self.items()),
            *(tuple(tensor2mu(k,v) for k,v in self.items()) if mu else ())
        ),**kwargs)

> Note: I think that BD should do zero undirected shae correction. I think it would be better for it to validate the shapes have batch dims
    that match. But I think that the __init__ should accept a shape_map for a key->single batch shape. I can have a default 
    key map so it can still be convenient, however this would open up BD to be more flexible.

In [24]:
TensorBatch.vstack((Tensor([[1,2,3,4]]),Tensor([[1,2,3,4]])))

TensorBatch([[1., 2., 3., 4.],
        [1., 2., 3., 4.]])

In [25]:
TensorBatch([1]).bs()

1

Ok so the solution was that `BD` itself does not validate or coerce batch sizes.
It does not check that they all match.
It merely uses the TensorBatch object in all its operations.
The TensorBatch object tracks and manages what the batch size is really supposed to be.

## Some Important Notes

In [26]:
## Numpy to Tensor Performance

In [27]:
img=np.random.randint(0,255,size=(240, 320, 3))

In [28]:
%%timeit
img=np.random.randint(0,255,size=(240, 320, 3))

1.59 ms ± 16.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [29]:
%%timeit
deepcopy(img)

298 µs ± 8.38 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [30]:
%%timeit
TensorBatch(img)

105 µs ± 8.59 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [31]:
%%timeit
TensorBatch([img])

/opt/conda/lib/python3.7/site-packages/fastai/torch_core.py:131: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:203.)
  else torch.tensor(x, **kwargs) if isinstance(x, (tuple,list))


140 ms ± 1.91 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


You will notice that if you wrap a numpy in a list, it completely kills the performance. The solution is to
just add a batch dim to the existing array and pass it directly.

In [32]:
%%timeit
TensorBatch(np.expand_dims(img,0))

96.2 µs ± 1.1 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In fact we can just test this with python lists...

In [33]:
%%timeit
TensorBatch([[1]])

92.4 µs ± 1.22 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [34]:
test_arr=[[1]*270000]

In [35]:
%%timeit
TensorBatch(test_arr)

14.8 ms ± 172 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [36]:
test_arr=np.array([[1]*270000])

In [37]:
%%timeit
TensorBatch(test_arr)

86.3 µs ± 1.63 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


This is horrifying just how made of a performance hit this causes... So we will be avoiding python list inputs 
to Tensors for now on...

## Testing
> Additional utilities for testing anything

In [38]:
# export
def test_in(a,b):
    "`test` that `a in b`"
    test(a,b,in_, ' in ')

In [39]:
test_in('o','hello')
test_in(3,[1,2,3,4])

In [40]:
# export
def _len_check(a,b): 
    return len(a)==(len(b) if not isinstance(b,int) else b)

def test_len(a,b):
    "`test` that `len(a) == int(b) or len(a) == len(b)`"
    test(a,b,_len_check, ' len == len ')

In [41]:
test_len([1,2,3],3)
test_len([1,2,3],[1,2,3])
test_len([1,2,3],'123')
test_fail(lambda:test_len([1,2,3],'1234'))

In [42]:
# export
def _less_than(a,b): return a < b
def test_lt(a,b):
    "`test` that `a < b`"
    test(a,b,_less_than, ' a < b')

In [43]:
test_lt(4,5)
test_fail(lambda:test_lt(5,4))

## Basic Visualization

In [44]:
# export
def run_tensorboard(port=6006, # The port to run tensorboard on/connect on
                    start_tag=None, # Starting regex e.g.: experience_replay/1
                    samples_per_plugin=None, # Sampling freq such as  images=0 (keep all)
                    extra_args=None, # Any additional arguments in the `--arg value` format
                    rm_glob=None # Remove old logs via a parttern e.g.: '*' will remove all files: runs/* 
                   ):
    if rm_glob is not None:
        for p in Path('runs').glob(rm_glob): p.delete()
    import socket
    from tensorboard import notebook
    a_socket=socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    cmd=None
    if not a_socket.connect_ex(('127.0.0.1',6006)):
        notebook.display(port=port,height=1000)
    else:
        cmd=f'--logdir runs --port {port} --host=0.0.0.0'
        if samples_per_plugin is not None: cmd+=f' --samples_per_plugin {samples_per_plugin}'
        if start_tag is not None:          cmd+=f' --tag {start_tag}'
        if extra_args is not None:         cmd+=f' {extra_args}'
        notebook.start(cmd)
    return cmd

In [45]:
# hide
SHOW_TENSOR_BOARD=False
if not os.environ.get("IN_TEST", None) and SHOW_TENSOR_BOARD:
    run_tensorboard()

In [51]:
# hide
from fastcore.imports import in_colab

# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.export import *
    from nbdev.export2html import *
    from nbdev.cli import *
    make_readme()
    notebook2script(silent=True)
    

converting /home/fastrl_user/fastrl/nbs/index.ipynb to README.md
